In [ ]:
from drought.visualization import ee_viz
import ee

In [ ]:
ee.Initialize()

In [ ]:
# ET and PET data for the first 6 months of 2019.
start_date = ee.Date('2019-01-01')
end_date = ee.Date('2019-01-06')\

ic = ee.ImageCollection('MODIS/006/MOD16A2') \
    .select(['ET', 'PET']) \
    .filterDate(start_date, end_date)


In [ ]:
# Visualize ET and PET for a single polygon.
ee_viz.viz_single_polygon(ic.first(), polygon_index=7, bands=["ET", "PET"], band_names=['Evapotranspiration','Potential Evepotranspiration'])

In [ ]:
ee_viz.viz_all_polygons(ic.mean(), bands=["ET", "PET"], band_names=['Evapotranspiration','Potential Evepotranspiration'])